# Reshaping

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
#Let's see how stack and unstack work

# Create DataFrame
dframe1 = DataFrame(np.arange(8).reshape((2, 4)),
                 index=pd.Index(['LA', 'SF'], name='city'),
                 columns=pd.Index(['A', 'B', 'C','D'], name='letter'))
#Show
dframe1

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [7]:
# Use stack to pivot the columns into the rows
dframe_st = dframe1.stack()

#Show
dframe_st

city  letter
LA    A         0
      B         1
      C         2
      D         3
SF    A         4
      B         5
      C         6
      D         7
dtype: int32

In [8]:
#We can always rearrange back into a DataFrame
dframe_st.unstack()

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [10]:
#We can choose which level to unstack by
dframe_st.unstack(0)

city,LA,SF
letter,,
A,0,4
B,1,5
C,2,6
D,3,7


In [12]:
# Also by which name to unstack by
dframe_st.unstack('letter')

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [13]:
# Also by which name to unstack by
dframe_st.unstack('city')

city,LA,SF
letter,,
A,0,4
B,1,5
C,2,6
D,3,7


In [15]:
# Let's see how stack and unstack handle NAN

#Make two series
ser1 = Series([0, 1, 2], index=['Q', 'X', 'Y'])
ser2 = Series([4, 5, 6], index=['X', 'Y', 'Z'])

#Concat to make a dframe
dframe = pd.concat([ser1, ser2], keys=['Alpha', 'Beta'])

# Unstack resulting DataFrame
dframe.unstack()

,Q,X,Y,Z
Alpha,0,1,2,NaN
Beta,NaN,4,5,6


In [16]:
# Now stack will filter out NAN by default
dframe.unstack().stack()

Alpha  Q    0
       X    1
       Y    2
Beta   X    4
       Y    5
       Z    6
dtype: float64

In [17]:
# IF we dont want this we can set it to False
dframe.unstack().stack(dropna=False)

Alpha  Q     0
       X     1
       Y     2
       Z   NaN
Beta   Q   NaN
       X     4
       Y     5
       Z     6
dtype: float64

# Pivoting

In [62]:
# Lets create some data to play with:

# Note: It is not necessary to understand how this dataset was made to understand this Lecture.

#import pandas testing utility
import pandas.util.testing as tm; tm.N = 3

#Create a unpivoted function
def unpivot(frame):
    N, K = frame.shape
    
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    
    # Return the DataFrame
    return DataFrame(data, columns=['date', 'variable', 'value'])

#Set the DataFrame we'll be using
dframe = unpivot(tm.makeTimeDataFrame())

In [63]:
#Show the "stacked" data, note how there are multiple variables and values for the dates
dframe

,date,variable,value
0,2000-01-03,A,-0.157500
1,2000-01-04,A,-0.200030
2,2000-01-05,A,1.395275
3,2000-01-03,B,0.553046
4,2000-01-04,B,0.393459
5,2000-01-05,B,0.176259
6,2000-01-03,C,1.652481
7,2000-01-04,C,1.645395
8,2000-01-05,C,0.311638
9,2000-01-03,D,-1.394883


In [68]:
# Now let's pivot the data

# First two value spassed are teh row and column indexes, then finally an optional fill value
dframe_piv = dframe.pivot('date','variable','value')

#Show
dframe_piv

variable,A,B,C,D
date,,,,
2000-01-03,-0.157500,0.553046,1.652481,-1.394883
2000-01-04,-0.200030,0.393459,1.645395,0.067484
2000-01-05,1.395275,0.176259,0.311638,-0.819208


# Cross-Tabulations

In [1]:
# THis will be a quick lesson on cross-tabulations, which are basically a special case of pivot-tables

In [3]:
# Let's create a quick data set
from StringIO import StringIO

data ="""\
Sample   Animal   Intelligence
1        Dog     Smart
2 Dog Smart
3 Cat Dumb
4 Cat Dumb
5 Dog Dumb
6 Cat Smart"""

#Store as dframe
dframe = pd.read_table(StringIO(data),sep='\s+')

In [4]:
# Show
dframe

,Sample,Animal,Intelligence
0,1,Dog,Smart
1,2,Dog,Smart
2,3,Cat,Dumb
3,4,Cat,Dumb
4,5,Dog,Dumb
5,6,Cat,Smart


In [6]:
# Now we can create a cross-tabulation table, which is basically just a frequency table
pd.crosstab(dframe.Animal,dframe.Intelligence,margins=True)

Intelligence,Dumb,Smart,All
Animal,,,
Cat,2,1,3
Dog,1,2,3
All,3,3,6


# Duplicates in DataFrame

In [7]:
#Lets get a dataframe with duplicates

dframe = DataFrame({'key1': ['A'] * 2 + ['B'] * 3,
                  'key2': [2, 2, 2, 3, 3]})

#Show
dframe

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [8]:
#We can use duplicated to find duplicates
dframe.duplicated()

0    False
1     True
2    False
3    False
4     True
dtype: bool

In [9]:
# We can also drop duplicates like this:
dframe.drop_duplicates()

,key1,key2
0,A,2
2,B,2
3,B,3


In [10]:
#You can filter which duplicates to drop by a single column
dframe.drop_duplicates(['key1'])

,key1,key2
0,A,2
2,B,2


In [11]:
#Show original
dframe

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [14]:
#By default the first value was taken for the duplicates, we can also take the last value instead
dframe.drop_duplicates(['key1'],take_last=True)

,key1,key2
1,A,2
4,B,3


In [ ]:
#Lets get a dataframe with duplicates

dframe = DataFrame({'key1': ['A'] * 2 + ['B'] * 3,
                  'key2': [2, 2, 2, 3, 3]})

#Show
dframe

#We can use duplicated to find duplicates
dframe.duplicated()

# We can also drop duplicates like this:
dframe.drop_duplicates()

#You can filter which duplicates to drop by a single column
dframe.drop_duplicates(['key1'])

#Show original
dframe

#By default the first value was taken for the duplicates, we can also take the last value instead
dframe.drop_duplicates(['key1'],take_last=True)

# Mapping

In [2]:
# Let's create a dframe to work with (Highest elevation cities in USA)
dframe = DataFrame({'city':['Alma','Brian Head','Fox Park'],
                    'altitude':[3158,3000,2762]})

#Show
dframe

,altitude,city
0,3158,Alma
1,3000,Brian Head
2,2762,Fox Park


In [6]:
#Now let's say we wanted to add a column for the States, we can do that with a mapping.
state_map={'Alma':'Colorado','Brian Head':'Utah','Fox Park':'Wyoming'}


In [7]:
# Now we can map that data to our current dframe
dframe['state'] = dframe['city'].map(state_map)

In [8]:
#Show result
dframe

,altitude,city,state
0,3158,Alma,Colorado
1,3000,Brian Head,Utah
2,2762,Fox Park,Wyoming


In [9]:
# Mapping is a great way to do element-wise transfomations and other data cleaning operations!

# Replacing Values

In [3]:
# Lets make  Series
ser1 = Series([1,2,3,4,1,2,3,4])
#Show
ser1

0    1
1    2
2    3
3    4
4    1
5    2
6    3
7    4
dtype: int64

In [4]:
# Using replace we can select --> .replace(value to be replaced, new_value)
ser1.replace(1,np.nan)

0   NaN
1     2
2     3
3     4
4   NaN
5     2
6     3
7     4
dtype: float64

In [5]:
#Can also input lists
ser1.replace([1,4],[100,400])

0    100
1      2
2      3
3    400
4    100
5      2
6      3
7    400
dtype: int64

In [15]:
#Can also input dictionary
ser1.replace({4:np.nan})

0     1
1     2
2     3
3   NaN
4     1
5     2
6     3
7   NaN
dtype: float64

# Rename Index

In [13]:
# Making a DataFrame
dframe= DataFrame(np.arange(12).reshape((3, 4)),
                 index=['NY', 'LA', 'SF'],
                 columns=['A', 'B', 'C', 'D'])

#Show
dframe

,A,B,C,D
NY,0,1,2,3
LA,4,5,6,7
SF,8,9,10,11


In [14]:
# Just like a Series, axis indexes can also use map

#Let's use map to lowercase the city initials
dframe.index.map(str.lower)

array(['ny', 'la', 'sf'], dtype=object)

In [25]:
# If you want to assign this to the actual index, you can use index
dframe.index = dframe.index.map(str.lower)
#Show
dframe

,A,B,C,D
ny,0,1,2,3
la,4,5,6,7
sf,8,9,10,11


In [28]:
# Use rename if you want to create a transformed version withour modifying the original!

#str.title will capitalize the first letter, lowercasing the columns
dframe.rename(index=str.title, columns=str.lower)


,a,b,c,d
Ny,0,1,2,3
La,4,5,6,7
Sf,8,9,10,11


In [34]:
# We can also use rename to insert dictionaries providing new values for indexes or columns!
dframe.rename(index={'ny': 'NEW YORK'},
            columns={'A': 'ALPHA'})

,ALPHA,B,C,D
NEW YORK,0,1,2,3
la,4,5,6,7
sf,8,9,10,11


In [38]:
# If you would like to actually edit the data set in place, set inplace=True
dframe.rename(index={'ny': 'NEW YORK'}, inplace=True)
dframe

,A,B,C,D
NEW YORK,0,1,2,3
la,4,5,6,7
sf,8,9,10,11


# Binning

In [2]:
#Now we'll learn about binning

In [3]:
years = [1990,1991,1992,2008,2012,2015,1987,1969,2013,2008,1999]

In [4]:
# We can seperate these years by decade
decade_bins = [1960,1970,1980,1990,2000,2010,2020]

In [7]:
#Now we'll use cut to get somethign called a Category object
decade_cat = pd.cut(years,decade_bins)

In [8]:
#Show
decade_cat

[(1980, 1990], (1990, 2000], (1990, 2000], (2000, 2010], (2010, 2020], ..., (1980, 1990], (1960, 1970], (2010, 2020], (2000, 2010], (1990, 2000]]
Length: 11
Categories (6, object): [(1960, 1970] < (1970, 1980] < (1980, 1990] < (1990, 2000] < (2000, 2010] < (2010, 2020]]

In [13]:
# We can check the categories using .categories
decade_cat.categories

Index([u'(1960, 1970]', u'(1970, 1980]', u'(1980, 1990]', u'(1990, 2000]', u'(2000, 2010]', u'(2010, 2020]'], dtype='object')

In [16]:
# Then we can check the value counts in each category
pd.value_counts(decade_cat)

(2010, 2020]    3
(1990, 2000]    3
(2000, 2010]    2
(1980, 1990]    2
(1960, 1970]    1
(1970, 1980]    0
dtype: int64

In [30]:
# We can also pass data values to the cut.

#For instance, if we just wanted to make two bins, evenly spaced based on max and min year, with a 1 year precision
pd.cut(years,2,precision=1)

[(1969, 1992], (1969, 1992], (1969, 1992], (1992, 2015], (1992, 2015], ..., (1969, 1992], (1969, 1992], (1992, 2015], (1992, 2015], (1992, 2015]]
Length: 11
Categories (2, object): [(1969, 1992] < (1992, 2015]]

In [1]:
# Thats about it for binning basics
# One last thing to note, jus tlike in standard math notation, when setting up bins:
# () means open, while [] means closed/inclusive

# Outlier

In [24]:
# Let's see how we would find outliers in a dataset

# First we'll seed the numpy generator
np.random.seed(12345)

#Next we'll create the dataframe
dframe = DataFrame(np.random.randn(1000,4))

In [25]:
#Show preview
dframe.head()

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570


In [26]:
# Lets describe the data
dframe.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [27]:
# Lets select the first column
col = dframe[0]

In [28]:
# NOw we can check which values in the column are greater than 3, for instance.
col[np.abs(col)>3]

523   -3.428254
900    3.366626
Name: 0, dtype: float64

In [29]:
# So we now know in column[0], rows 523 and 900 have values with abs > 3

#How about all the columns?

# We can use the "any" method
dframe[(np.abs(dframe)>3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [33]:
# WE could also possibly cap the data at 3

dframe[np.abs(dframe)>3] = np.sign(dframe) *3

In [34]:
dframe.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.061623,0.074473,0.037153,0.009919
std,0.995875,0.989820,1.003604,0.989688
min,-2.969411,-2.989741,-2.925113,-2.881858
25%,-0.774132,-0.588138,-0.622310,-0.636641
50%,-0.115171,0.102787,0.012889,-0.010997
75%,0.619779,0.787953,0.682401,0.659019
max,3.000000,3.000000,3.000000,3.000000
